In [1]:
# Import required libraries
import pandas as pd
import ibis
from ibis import _, selectors as s
import numpy as np
import altair as alt
from pathlib import Path
import sys

# Import project modules
from process import (
    process_isora,
    process_wb,
    process_gfi,
    process_usaid,
    process_fsi,
    process_unodc,
    clean_nexus_data
)
from config import ISORA_FILES, WDI_INDICATOR_CODES, WGI_INDICATOR_LABELS, GFI_CONFIG

# Configure Altair for rendering larger datasets
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

## Utils

In [2]:
# Setup project paths
PROJECT_ROOT = Path.cwd()  # Assuming notebook is in a notebooks/ subdirectory
DATA_DIR = PROJECT_ROOT / "data"
RAW_DATA_PATH = DATA_DIR / "raw"
PROCESSED_DATA_PATH = DATA_DIR / "processed"

# Add project root to Python path for imports
sys.path.append(str(PROJECT_ROOT))

# Functions
def dt_map(df, code=True):
    base_cols = ['source', 'database', 'collection']
    cols = base_cols + ['indicator_code'] if code else base_cols
    return (
        df
        .filter(cols)
        .value_counts()
        .reset_index(name='count')
        .sort_values(by=cols+['count'])
    )

def src_tbl(df, ind='code'):
    ind_col = "indicator_label" if ind == "label" else "indicator_code"
    return (
        df
        .filter(['country','year',ind_col,'value'])
        .pivot_table(
            index=['country', 'year'],
            columns=ind_col,
            values='value'
        )
    )

## Load Individual Sources

In [8]:
# Process each data source
pefa, taxwb, wb_nexus = process_wb(RAW_DATA_PATH)
isora, gfi, usaid, fsi, unodc = (
    globals()[f"process_{name}"](RAW_DATA_PATH) for name in ["isora", "gfi", "usaid", "fsi", "unodc"]
) 

## Load NEXUS

In [3]:
nexus=pd.read_parquet(PROCESSED_DATA_PATH / "nexus.parquet")

## Working with sources

In [24]:
src_tbl(pefa).describe()

indicator_code,WB.PEFA.PI-2011-01,WB.PEFA.PI-2011-02,WB.PEFA.PI-2011-02i,WB.PEFA.PI-2011-02ii,WB.PEFA.PI-2011-03,WB.PEFA.PI-2011-04,WB.PEFA.PI-2011-04i,WB.PEFA.PI-2011-04ii,WB.PEFA.PI-2011-05,WB.PEFA.PI-2011-06,...,WB.PEFA.PI-2016-30,WB.PEFA.PI-2016-30.1,WB.PEFA.PI-2016-30.2,WB.PEFA.PI-2016-30.3,WB.PEFA.PI-2016-30.4,WB.PEFA.PI-2016-31,WB.PEFA.PI-2016-31.1,WB.PEFA.PI-2016-31.2,WB.PEFA.PI-2016-31.3,WB.PEFA.PI-2016-31.4
count,186.000000,185.000000,185.000000,78.000000,186.000000,184.000000,175.000000,173.000000,186.000000,185.000000,...,65.000000,63.000000,63.000000,62.000000,65.000000,59.000000,59.000000,58.000000,59.000000,59.000000
mean,2.741935,2.340541,2.178378,3.730769,3.053763,2.296196,2.520000,2.312139,2.854839,3.070270,...,2.023077,2.285714,2.523810,2.258065,2.338462,2.076271,2.101695,1.965517,2.050847,2.135593
std,1.054240,1.038775,1.071147,0.767405,1.113704,1.129842,1.312364,1.159251,0.933116,0.897116,...,0.772732,1.069045,1.175793,0.974015,1.189497,0.959587,1.227486,1.075493,1.057383,1.238153
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,1.500000,1.000000,4.000000,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,...,1.500000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,3.000000,2.000000,2.000000,4.000000,3.000000,2.000000,2.000000,2.000000,3.000000,3.000000,...,1.500000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,4.000000,3.000000,3.000000,4.000000,4.000000,3.500000,4.000000,3.000000,4.000000,4.000000,...,2.500000,3.000000,3.500000,3.000000,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [23]:
src_tbl(pefa,"label").describe()

indicator_label,PEFA: Accounting standards,PEFA: Accounting standards used,PEFA: Accuracy of in-year budget reports,PEFA: Adequacy of time for the legislature to provide a response to budget proposals .,PEFA: Advance accounts,PEFA: Aggregate expenditure out-turn,PEFA: Aggregate expenditure out-turn compared to original approved budget,PEFA: Aggregate revenue out-turn compared to original approved budget,PEFA: Agregate revenue outturn,PEFA: Alignment of strategic plans and medium-term budgets,...,PEFA: Timing of budget approval,PEFA: Timing of in-year budget reports,PEFA: Transfer of revenue collections,PEFA: Transparency of asset disposal,PEFA: Transparency of inter-governmental fiscal relations,PEFA: Transparency of legislative scrutiny of audit reports,PEFA: Transparency of taxpayer obligations and liabilities,"PEFA: Transparency, comprehensiveness and competition in the legal and regulatory framework.",PEFA: Transparent and rules based systems in the horizontal allocation,PEFA: Use of competitive procurement methods.
count,65.000000,176.000000,63.000000,173.000000,64.000000,65.000000,186.000000,186.000000,65.000000,65.000000,...,65.000000,63.000000,65.000000,65.000000,161.000000,59.000000,180.000000,176.000000,155.000000,176.000000
mean,2.030769,2.278409,2.349206,2.930636,2.203125,2.723077,2.741935,3.053763,2.484615,1.907692,...,3.415385,2.079365,3.615385,1.953846,2.739130,2.135593,2.936111,2.454545,3.161290,2.136364
std,0.809499,0.977922,0.935996,1.159309,1.210794,1.192525,1.054240,1.113704,1.198858,0.979845,...,0.966506,1.067127,0.654129,0.891466,0.937228,1.238153,0.728568,1.051900,1.136661,1.092121
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,...,3.000000,1.000000,3.000000,1.000000,2.000000,1.000000,2.500000,1.000000,2.000000,1.000000
50%,2.000000,2.000000,2.000000,3.000000,2.000000,3.000000,3.000000,3.000000,3.000000,2.000000,...,4.000000,2.000000,4.000000,2.000000,3.000000,2.000000,3.000000,3.000000,4.000000,2.000000
75%,2.000000,3.000000,3.000000,4.000000,3.000000,4.000000,4.000000,4.000000,4.000000,2.000000,...,4.000000,3.000000,4.000000,2.000000,3.500000,3.000000,3.500000,3.000000,4.000000,3.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [27]:
src_tbl(pefa,"label").info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 250 entries, ('AFG', '2005') to ('ZWE', '2018')
Columns: 218 entries, PEFA: Accounting standards to PEFA: Use of competitive procurement methods.
dtypes: float64(218)
memory usage: 431.4+ KB


In [45]:
dt_map(pefa)

,source,database,collection,indicator_code,count
0,World Bank,WB-PEFA.xlsx,PEFA,WB.PEFA.PI-2011-01,1683
21,World Bank,WB-PEFA.xlsx,PEFA,WB.PEFA.PI-2011-02,1683
44,World Bank,WB-PEFA.xlsx,PEFA,WB.PEFA.PI-2011-02i,1683
91,World Bank,WB-PEFA.xlsx,PEFA,WB.PEFA.PI-2011-02ii,1241
52,World Bank,WB-PEFA.xlsx,PEFA,WB.PEFA.PI-2011-03,1683
...,...,...,...,...,...
211,World Bank,WB-PEFA.xlsx,PEFA,WB.PEFA.PI-2016-31,935
212,World Bank,WB-PEFA.xlsx,PEFA,WB.PEFA.PI-2016-31.1,935
213,World Bank,WB-PEFA.xlsx,PEFA,WB.PEFA.PI-2016-31.2,935
214,World Bank,WB-PEFA.xlsx,PEFA,WB.PEFA.PI-2016-31.3,935


In [44]:
dt_map(isora, False)

,source,database,collection,count
17,ISORA,IMF ISORA.xlsx,Electronic filing rates by tax,1485
13,ISORA,IMF ISORA.xlsx,On-time filing rates by tax typ,1980
4,ISORA,IMF ISORA.xlsx,Percentage inactive taxpayers o,2475
5,ISORA,IMF ISORA.xlsx,Proportion of returns by ch_0,2475
6,ISORA,IMF ISORA.xlsx,Proportion of returns by ch_1,2475
7,ISORA,IMF ISORA.xlsx,Proportion of returns by channe,2475
20,ISORA,IMF ISORA.xlsx,Registration of personal income,990
0,ISORA,IMF ISORA.xlsx,Segmentation ratios LTO or prog,2970
19,ISORA,"imf isora op metrics audit, criminal investiga...",Audit and verification,990
14,ISORA,"imf isora op metrics audit, criminal investiga...",Dispute resolution review proce,1980


## Working with Nexus OBT

In [6]:
nexus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616409 entries, 0 to 616408
Data columns (total 34 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   year                                      616409 non-null  Int64  
 1   value                                     582793 non-null  float64
 2   source                                    616409 non-null  object 
 3   indicator_code                            616409 non-null  object 
 4   indicator_label                           616409 non-null  object 
 5   database                                  616409 non-null  object 
 6   collection                                616409 non-null  object 
 7   value_meta                                33928 non-null   object 
 8   global_code                               603777 non-null  float64
 9   global_name                               603777 non-null  object 
 10  region_code         

In [ ]:
dt_map(nexus)

In [37]:
src_tbl(nexus.query('source=="ISORA"'))

indicator_code  Academic qualifications (No. of staff at the end of FY)-Bachelors degree  \
country year                                                                               
ABW     2020                                                 26.0                          
AFG     2018                                               2145.0                          
        2019                                               2145.0                          
AGO     2018                                               1848.0                          
        2019                                               1923.0                          
...                                                           ...                          
ZMB     2019                                                  NaN                          
        2020                                               1443.0                          
ZWE     2018                                               1658.0                          
        2019                                               1578.0                          
        2020                                               1997.0                          

indicator_code  Academic qualifications (No. of staff at the end of FY)-Masters degree (or above)  \
country year                                                                                        
ABW     2020                                                 18.0                                   
AFG     2018                                                120.0                                   
        2019                                                120.0                                   
AGO     2018                                                 76.0                                   
        2019                                                 83.0                                   
...                                                           ...                                   
ZMB     2019                                                  NaN                                   
        2020                                                136.0                                   
ZWE     2018                                                514.0                                   
        2019                                                496.0                                   
        2020                                                411.0                                   

indicator_code  Active taxpayers on PIT register as percentage of Labor Force  \
country year                                                                    
ABW     2020                                           101.553445               
AFG     2018                                             0.615603               
        2019                                             0.709346               
AGO     2018                                            37.297847               
        2019                                            39.828213               
...                                                           ...               
ZMB     2019                                                  NaN               
        2020                                             0.176075               
ZWE     2018                                             0.350737               
        2019                                             0.645954               
        2020                                             0.000200               

indicator_code  Active taxpayers on PIT register as percentage of Population  \
country year                                                                   
ABW     2020                                            54.984733              
AFG     2018                                             0.170895              
        2019                                             0.199547              
AGO     2018                       

## Start exploring the sources and nexus